In [69]:
from pathlib import Path
import pandas as pd
import numpy as np
import glob as gb

In [70]:
PERF_DIR = Path('/Users/julien/Software/Others/OS-build-release/results/')

In [71]:
csvs = gb.glob(str(PERF_DIR / '*.csv'))

In [72]:
csv = csvs[0]

In [73]:
def find_skip_rows(results_file: Path) -> int:
    """
    Open the result benchmark, and locate the actual start of the timings

    Returns:
    --------
    skiprows (int): the number of lines to skip when read_csv is called
    """
    skiprows = 0
    search_str = 'name,iterations,'
    with open(results_file, 'r') as f:
        content = f.read()
    if search_str not in content:
        return None
    lines = content.splitlines()
    while (search_str not in lines[skiprows]):
        skiprows += 1

    return skiprows

In [74]:
def read_bench_results(results_file: Path) -> pd.DataFrame:
    skiprows = find_skip_rows(csv)
    if skiprows is None:
        return None
    df = pd.read_csv(csv, skiprows=skiprows, index_col=0)
    df = df.loc[df['iterations'].notnull()]
    # Convert everything in ms
    for col in ['real_time', 'cpu_time']:
        df[col] = (df[[col, 'time_unit']].apply(
            lambda row: pd.to_timedelta(arg=row[0], unit=row[1]), axis=1)
                   .dt.total_seconds() * 1e3)
        
    return df

In [75]:
df_dict = {}
dfs = []
for csv in csvs:
    fname = Path(csv).name
    #if 'Model_ModelObjects' in fname:
    #    fname = 'Model_ModelObjects'
    prefix  = Path(csv).name.split('_')[0]
    test = '_'.join(fname.split('_')[1:-1])
    
    df = read_bench_results(csv)
    if df is None:
        continue
    df['test_file'] = test
    df['prefix'] = prefix
    dfs.append(df)

In [76]:
df = pd.concat(dfs, axis=0)

In [77]:
df.set_index(['prefix', 'test_file'], append=True, inplace=True)

In [78]:
df_real = df['real_time'].unstack('prefix')

In [79]:
df_real.loc[[x for x in df_real.index if 'VT' in x[0]]]

,prefix,Model,move1,move2,move3,moveAll,new,new2,newvt1,newvt2,newvt3,ori,ori2
name,test_file,,,,,,,,,,,,
BM_VT/CONTAMTemplate,VersionTranslation,NaN,1520.04000,1510.71000,1519.58000,1494.54000,1500.82000,1516.13000,1508.49000,1526.08000,1546.49000,NaN,NaN
BM_VT/Model12,VersionTranslation,NaN,1332.44000,1335.87000,1351.26000,1323.00000,1327.51000,1339.62000,1345.44000,1342.55000,1331.28000,NaN,NaN
BM_VT/Windows_Complete,VersionTranslation,NaN,2121.90000,2177.60000,2139.45000,2113.83000,2108.60000,2126.18000,2134.04000,2136.61000,2141.24000,NaN,NaN
BM_VT/example_1_13_4,VersionTranslation,NaN,3.46746,3.53194,3.47998,3.49135,3.47885,3.49547,3.72791,3.49230,3.69145,NaN,NaN
BM_VT/example_1_14_0,VersionTranslation,NaN,3.44740,3.54888,3.47124,3.50161,3.49487,3.48802,3.51183,3.49641,3.70009,NaN,NaN
BM_VT/floorplan_school,VersionTranslation,NaN,24.93580,25.32990,25.43590,24.43610,25.83320,26.88220,26.36430,25.22210,25.31760,NaN,NaN
BM_VT/seb,VersionTranslation,NaN,3.45500,3.49150,3.54873,3.46416,3.47192,3.49553,3.55374,3.53420,3.61637,NaN,NaN


In [80]:
df_real[df_real['move1'].isnull()].index.tolist()

[('BM_AddObjects/8', ''),
 ('BM_LoadIdfFile/CONTAMTemplate', 'VersionTranslation'),
 ('BM_LoadIdfFile/Model12', 'VersionTranslation'),
 ('BM_LoadIdfFile/Windows_Complete', 'VersionTranslation'),
 ('BM_LoadIdfFile/example_1_13_4', 'VersionTranslation'),
 ('BM_LoadIdfFile/example_1_14_0', 'VersionTranslation'),
 ('BM_LoadIdfFile/floorplan_school', 'VersionTranslation'),
 ('BM_LoadIdfFile/seb', 'VersionTranslation'),
 ('BM_LoadIdfFile/two_stories_pre_intersect', 'VersionTranslation')]

In [81]:
df_real.sum(axis=0).sort_values()

prefix
Model          6.035270
moveAll    48285.479184
move1      49511.417595
move2      49840.202271
new        50048.740669
move3      50092.161187
ori        50792.795020
new2       50833.869913
ori2       51210.343418
newvt1     51376.224483
newvt3     51753.438984
newvt2     52844.690581
dtype: float64

In [82]:
df_sum = df_real[df_real['move2'].notnull()].groupby('test_file').sum()
df_sum['orim'] = df_sum[['ori', 'ori2']].mean(axis=1)
df_sum[['orim', 'move1', 'move2', 'move3', 'moveAll']]

prefix,orim,move1,move2,move3,moveAll
test_file,,,,,
Checksum,7.891925,7.862002,7.948927,7.909946,7.845993
ForwardTranslator,23687.312450,23440.157499,23434.204000,23707.347300,22634.379800
IddObjectParse,0.232607,0.230253,0.221959,0.230955,0.224620
IdfObjectParse,0.032448,0.031735,0.032742,0.031871,0.031633
LoadIdd,130.686900,129.918800,132.902400,131.963300,127.521900
LoadIdfFile,208.431105,208.479790,204.556010,211.076030,207.877439
Model,17281.876215,16597.290800,16862.121380,16813.563290,16274.992440
Model_ModelObjects,0.324056,0.327162,0.326187,0.327322,0.314030
ThermalZoneCombineSpaces,2515.377631,2122.531485,2100.610935,2113.131423,2069.671310


In [68]:
df_sum[['orim', 'move3']].groupby('test_file').sum().pct_change(axis=1).iloc[:, [-1]].style.format('{:.2%}')

prefix,move3
test_file,
Checksum,0.23%
ForwardTranslator,0.08%
IddObjectParse,-0.71%
IdfObjectParse,-1.78%
LoadIdd,0.98%
LoadIdfFile,1.27%
Model,-2.71%
Model_ModelObjects,1.01%
ThermalZoneCombineSpaces,-15.99%


In [51]:
df_sum[['orim', 'move2']].groupby('test_file').sum().pct_change(axis=1).iloc[:, [-1]].style.format('{:.2%}')

prefix,move2
test_file,
Checksum,0.72%
ForwardTranslator,-1.07%
IddObjectParse,-4.58%
IdfObjectParse,0.91%
LoadIdd,1.70%
LoadIdfFile,-1.86%
Model,-2.43%
Model_ModelObjects,0.66%
ThermalZoneCombineSpaces,-16.49%


In [ ]:
df_sum['move1'].subtract(df_sum[['ori', 'ori2']].mean(axis=1))

In [ ]:
df_real[['ori', 'move1']].groupby('test_file').sum().pct_change(axis=1).iloc[:, [-1]].style.format('{:.2%}')

In [ ]:
df_real.plot(figsize=(16, 9))

In [ ]:
df_real['orim'] = df_real[['ori', 'ori2']].mean(axis=1)
df_real['newm'] = df_real[['new'
                           #, 'new2'
                          ]].mean(axis=1)

In [ ]:
(df_real[['orim', 'newm']]).pct_change(axis=1)['newm'].sort_values().plot(kind='barh', figsize=(16, 36))

In [ ]:
(df_real['newm'] - df_real['orim']).sort_values().plot(kind='barh', figsize=(16, 9))

In [ ]:
df_real[['orim', 'newm']].plot(kind='barh', figsize=(16, 36))

In [ ]:
df_real[['orim', 'newm']].mean()